YOUNESS MOUAD (20 132 250)<br>
EL MAHDI ZOUHAIR (20 132 241)


In [11]:
import numpy as np
import gestion_donnees as gd

In [52]:
parametres = {}
parametres['regression_logistique'] = {'C': [8, 10, 15, 20, 25],
                                       'solver': ['newton-cg', 'lbfgs', 'liblinear']}

parametres['arbre_decision'] = {'criterion': ['gini', 'entropy'],
                                'min_samples_split': [2, 3, 4, 5]}

parametres['foret_arbres_decision'] = {'criterion': ['gini', 'entropy'],
                                       'n_estimators': [100, 150, 200],
                                       'min_samples_split': [2, 3, 4, 5]}

parametres['ada_boost'] = {'learning_rate': [2, 3, 4, 5]}

parametres['machine_vecteur_support'] = {'C': [40, 50, 60, 70],
                                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
                                         'degree': [2, 3, 4],
                                         'probability': [True]}

In [59]:
donnees = gd.GestionDonnees(True, False)

In [60]:
x_entrain, y_entrain, x_test, y_test, x_nouveau_test = donnees.generer_donnees()

In [25]:
from test import classifier_donnees

In [53]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

regression_logistique = LogisticRegression(max_iter=1000)
arbre_decision = DecisionTreeClassifier()
foret_arbres_decision = RandomForestClassifier()
machine_vecteur_support = SVC()

In [33]:
resultat_regression_logistique = classifier_donnees(regression_logistique, 
                                                    parametres['regression_logistique'], 
                                                    x_entrain, y_entrain, x_test, y_test)

In [34]:
resultat_arbre_decision = classifier_donnees(arbre_decision, 
                                             parametres['arbre_decision'], 
                                             x_entrain, y_entrain, x_test, y_test)

In [50]:
resultat_foret_arbres_decision = classifier_donnees(foret_arbres_decision, 
                                                    parametres['foret_arbres_decision'], 
                                                    x_entrain, y_entrain, x_test, y_test)

In [ ]:
resultat_machine_vecteur_support = classifier_donnees(machine_vecteur_support, 
                                                      parametres['machine_vecteur_support'], 
                                                      x_entrain, y_entrain, x_test, y_test)

In [35]:
resultat_regression_logistique

{'justess_entrain': 1.0,
 'justess_test': 0.9797979797979798,
 'meilleure_justesse': 0.9860918716662684,
 'meilleurs_params': {'C': 8, 'solver': 'newton-cg'},
 'meilleur_classifieur': LogisticRegression(C=8, max_iter=1000, solver='newton-cg')}

In [36]:
resultat_arbre_decision

{'justess_entrain': 0.9823232323232324,
 'justess_test': 0.5858585858585859,
 'meilleure_justesse': 0.611097842528461,
 'meilleurs_params': {'criterion': 'gini', 'min_samples_split': 3},
 'meilleur_classifieur': DecisionTreeClassifier(min_samples_split=3)}

In [51]:
resultat_foret_arbres_decision

{'justess_entrain': 1.0,
 'justess_test': 0.050505050505050504,
 'meilleure_justesse': 0.0669373457527267,
 'meilleurs_params': {'criterion': 'gini',
  'min_samples_split': 3,
  'n_estimators': 200},
 'meilleur_classifieur': RandomForestClassifier(min_samples_split=3, n_estimators=200)}

In [58]:
resultat_machine_vecteur_support


Justesses des classifieurs avec bruit = False et distribution gaussienne = False.
